In [138]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [139]:
!pip install accelerate==0.21.0 transformers==4.31.0 tokenizers==0.13.3
!pip install bitsandbytes==0.40.0 einops==0.6.1
!pip install xformers==0.0.22.post7
!pip install langchain==0.1.4
!pip install faiss-gpu==1.7.1.post3
!pip install sentence_transformers
!pip install dotenv

  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.3.1 requires transformers<5.0.0,>=4.32.0, but you have transformers 4.31.0 which is incompatible.


  Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install python-dotenv


In [1]:

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.huggingface_hub import HuggingFaceHub
from langchain.memory import ConversationBufferWindowMemory
import os
import re
from hashlib import blake2b
from tempfile import NamedTemporaryFile

import dotenv
import torch

from torch import cuda, bfloat16
import transformers
from transformers import StoppingCriteria, StoppingCriteriaList
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.retrievers.multi_query import MultiQueryRetriever



In [2]:
output_parent_folder="/content/drive/MyDrive/llama-2"


In [11]:
HUGGINGFACEHUB_API_TOKEN = "Your API Key"

# Set the environment variable
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

# Now you can access it
hf_auth = os.environ['HUGGINGFACEHUB_API_TOKEN']


In [3]:

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
# set quantization configuration to load large model with less GPU memory # this requires the `bitsandbytes`
bnb_config = transformers.BitsAndBytesConfig( load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=bfloat16 )
# begin initializing HF items, you need an access token

model_config = transformers.AutoConfig.from_pretrained( model_id, use_auth_token=hf_auth )
model = transformers.AutoModelForCausalLM.from_pretrained( model_id, trust_remote_code=True, config=model_config, quantization_config=bnb_config, device_map='auto', use_auth_token=hf_auth )
# enable evaluation mode to allow model inference
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1085: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained( model_id, use_auth_token=hf_auth )
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
import torch
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:711: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

In [5]:
 # define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
    for stop_ids in stop_token_ids:
      if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
        return True
    return False
stopping_criteria = StoppingCriteriaList([StopOnTokens()])
generate_text = transformers.pipeline( model=model, tokenizer=tokenizer, return_full_text=True,
                                      task='text-generation',  stopping_criteria=stopping_criteria,  temperature=0.1, max_new_tokens=512,  repetition_penalty=1.1 )


In [6]:
from langchain.llms import HuggingFacePipeline

chat = HuggingFacePipeline(pipeline=generate_text) # checking again that everything is working fine
memory=ConversationBufferWindowMemory(k=4,return_messages=True)


In [9]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template('''Character: Assume you are a chatbot that provides help users undestand various guidelines about the disease
Context: {context}. Previous user queries and your reply to them given here [{chat_history}]. Use these information only when in the question there are unspecified references.
Action: Question: {question} Provide reply without mentioning about the text feeded to you.
Output Format: Provide answer in human readable format if possible use emojis to better connect with users''')
)

In [7]:
query="List some of the infections that can be caused by hiv?";

In [12]:
%run '/content/drive/MyDrive/Colab Notebooks/rag_implementation.py'
# Specify the output folder path
rag_instance = RagImplementation(output_parent_folder)

In [13]:
retriever1=rag_instance.process(query)

In [ ]:
"""vector_store1 = FAISS.from_documents(documents, embedding_function)
retriever1 = vector_store1.as_retriever()
multi_query_retriever1 = MultiQueryRetriever.from_llm(retriever=retriever1, llm=llm)
relevant_documents = multi_query_retriever1.get_relevant_documents(query)
"""

In [ ]:
prompt.format(question=query,context="hello",chat_history=memory.buffer_as_str)

'Character: Assume you are a chatbot that provides help users undestand various guidelines about the disease\nContext: hello. Previous user queries and your reply to them given here []. Use these information only when in the question there are unspecified references. \nAction: Question: List some of the infections that can be caused by hiv? Provide reply without mentioning about the text feeded to you.\nOutput Format: Provide answer in human readable format if possible use emojis to better connect with users'

In [ ]:
memory

ConversationBufferWindowMemory(return_messages=True, k=4)

In [14]:
from langchain.chains import ConversationalRetrievalChain,LLMChain
chat_history = []

chain = ConversationalRetrievalChain.from_llm(chat, retriever1,combine_docs_chain_kwargs={"prompt": prompt}, return_source_documents=False, get_chat_history=lambda h : h,verbose=False)

#chain = LLMChain(llm=chatbot, prompt=prompt)
#chain.run(question=query,chat_history=chat_history)


result = chain({"question": query, "chat_history":memory})  # Include prompt_template in input
memory.save_context({"input": query}, {"output": result['answer']})

#result = chain({"question": query, "chat_history": chat_history})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [15]:
result

{'question': 'List some of the infections that can be caused by hiv?',
 'chat_history': ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='List some of the infections that can be caused by hiv?'), AIMessage(content=".\nAnswer: 🤕 Common infections that can be caused by HIV include opportunistic infections such as pneumocystis pneumonia (PCP), tuberculosis (TB), and cryptococcal meningitis. These infections can cause a wide range of symptoms, including fever, fatigue, night sweats, and weight loss. It's important for people living with HIV to receive regular medical care and take antiretroviral therapy (ART) to manage their virus and prevent these infections from occurring. 💊")]), return_messages=True, k=4),
 'answer': ".\nAnswer: 🤕 Common infections that can be caused by HIV include opportunistic infections such as pneumocystis pneumonia (PCP), tuberculosis (TB), and cryptococcal meningitis. These infections can cause a wide range of symptoms, 